# OpenAI queries with tools (functions)
### - provide OpenAI API with different functions to call in addition to its own knowledge base
- Wolfram Alpha API: provide OpenAI access to real-time data, mathematical and scientific knowledge
- Talk to your own database with the help of chatgpt

In [60]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMMathChain
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

from dotenv import load_dotenv
import os

# load your API key to the environment variables
load_dotenv()
API_KEY = os.getenv('OPENAI_API_KEY')

In [61]:
import wolframalpha
import ssl
import requests
import certifi

# workaround for mac to solve SSL: CERTIFICATE_VERIFY_FAILED Error
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()
os.environ["SSL_CERT_FILE"] = certifi.where()

In [62]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# provide OpenAI access to real-time data, mathematical and scientific knowledge
# llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
wolfram = WolframAlphaAPIWrapper()

# provide OpenAI access to data in specific database (in this case: kaggle Amazon 
# database about customers' reviews and their helpfulness)
db = SQLDatabase.from_uri("sqlite:///data/database.sqlite")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)

In [63]:
tools = [
    Tool(
        name="Calculator",
        #func=llm_math_chain.run,
        func=wolfram.run,
        description="useful for when you need to answer questions about real time data, mathematical computations or science problems"
    ),
    Tool(
        name="kaggle_amazon_db",
        func=db_chain.run,
        description="useful for when you need to answer questions about amazon customers' \
            reviews and their helpfulness. Input should be in the form of a question containing full context"
    )
]

In [64]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [66]:
answer = agent.run("What is the name of the user who wrote the largest number of helpful reviews for amazon?")



> Entering new AgentExecutor chain...

Invoking: `kaggle_amazon_db` with `What is the name of the user who wrote the largest number of helpful reviews for amazon?`




> Entering new SQLDatabaseChain chain...
What is the name of the user who wrote the largest number of helpful reviews for amazon?
SQLQuery:SELECT "UserId", COUNT(*) AS "HelpfulReviews"
FROM "Reviews"
WHERE "HelpfulnessNumerator" > 0
GROUP BY "UserId"
ORDER BY "HelpfulReviews" DESC
LIMIT 1;
SQLResult: [('A1YUL9PCJR3JTY', 364)]
Answer:A1YUL9PCJR3JTY
> Finished chain.
A1YUL9PCJR3JTYThe user with the profile name "A1YUL9PCJR3JTY" wrote the largest number of helpful reviews for Amazon.

> Finished chain.


In [38]:
try:
    answer = agent.run("What is 2x+5 = -3x + 7?")
except Exception as e:
    with open('error_out.txt', 'w') as out_file:
        out_file.write(str(e))



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `solve(2*x + 5 = -3*x + 7, x)`


Assumption: solve 2 x + 5 = -3 x + 7 for x 
Answer: x = 2/5The solution to the equation 2x + 5 = -3x + 7 is x = 2/5.

> Finished chain.


In [45]:
answer = agent.run("How far is it from Chicago to Tokyo?")
print(answer)



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `distance from Chicago to Tokyo`


Assumption: distance | from | Chicago, Illinois, United States
to | Tokyo, Japan 
Answer: 10160 km (kilometers)The distance from Chicago, Illinois, United States to Tokyo, Japan is approximately 10,160 kilometers.

> Finished chain.
The distance from Chicago, Illinois, United States to Tokyo, Japan is approximately 10,160 kilometers.


In [41]:
answer = agent.run("Make a picture of an airy function using the Calculator tool.")



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `airy(x)`


Assumption: Ai(x) 
Answer: integral Ai(x) dx = -(x (3^(1/3) x Γ(2/3)^2 _1 F_2(2/3 ;4/3, 5/3 ;x^3/9) - 3 Γ(1/3) Γ(5/3) _1 F_2(1/3 ;2/3, 4/3 ;x^3/9)))/(9 3^(2/3) Γ(2/3) Γ(4/3) Γ(5/3)) + constantI apologize, but I am unable to generate a picture of the airy function using the Calculator tool. However, I can provide you with the mathematical expression of the airy function:

Ai(x) = -(x (3^(1/3) x Γ(2/3)^2 _1 F_2(2/3 ;4/3, 5/3 ;x^3/9) - 3 Γ(1/3) Γ(5/3) _1 F_2(1/3 ;2/3, 4/3 ;x^3/9)))/(9 3^(2/3) Γ(2/3) Γ(4/3) Γ(5/3)) + constant

Please note that Γ represents the gamma function and _1 F_2 represents the hypergeometric function.

> Finished chain.


## use the wolframalpha library directly to get access to detailed parameters

- the langchain WolframAlphaAPIWrapper does not give access to request parameters or detailed response of the api
- by tuning the request parameters it is  possible to get images from the wolfram alpha API like wit the chatgpt plus wolfram alpha plugin

In [47]:
client = wolframalpha.Client(os.getenv('WOLFRAM_ALPHA_APPID'))
  
res = client.query('How far is it from Chicago to Tokyo?')
  
answer = next(res.results)
print(answer)

{'@title': 'Result', '@scanner': 'Data', '@id': 'Result', '@position': 200.0, '@error': 'false', '@numsubpods': 1, '@primary': True, 'subpod': {'@title': '', 'microsources': {'microsource': 'CityData'}, 'img': {'@src': 'https://www6b3.wolframalpha.com/Calculate/MSP/MSP199023hfc8f974e9e3ed000061gg06h0255afh42?MSPStoreType=image/gif&s=14', '@alt': '10160 km (kilometers)', '@title': '10160 km (kilometers)', '@width': 142, '@height': 19, '@type': 'Default', '@themes': '1,2,3,4,5,6,7,8,9,10,11,12', '@colorinvertable': 'true', '@contenttype': 'image/gif'}, 'plaintext': '10160 km (kilometers)'}, 'expressiontypes': {'@count': '1', 'expressiontype': {'@name': 'Default'}}}


### make direct request without any wolfram alpha library

In [50]:
function = "sin x cos y"
wa_appid = os.getenv('WOLFRAM_ALPHA_APPID')
query = f"plot {function}"
query_url = f"http://api.wolframalpha.com/v2/query?" \
            f"appid={wa_appid}" \
            f"&input={query}" \
            f"&output=json" \
            f"&includepodid=3DPlot" \
            f"&includepodid=ContourPlot"

r = requests.get(query_url).json()

In [55]:
import requests
from IPython.display import Image, display

image_url = r["queryresult"]["pods"][0]["subpods"][0]["img"]["src"]
#r2 = requests.get(image_url)

Image(url=image_url,format='gif')

In [56]:
function = "How far is the distance between Chicago and Tokyo?"
wa_appid = os.getenv('WOLFRAM_ALPHA_APPID')
query = f"plot {function}"
query_url = f"http://api.wolframalpha.com/v2/query?" \
            f"appid={wa_appid}" \
            f"&input={query}" \
            f"&output=json" \
            f"&includepodid=3DPlot" \
            f"&includepodid=ContourPlot"

r = requests.get(query_url).json()

In [58]:
try:
    #image_url2 = r["queryresult"]["pods"][0]["subpods"][0]["img"]["src"]
    pods = r["queryresult"].get('pods', None)
    if pods:
        image_url2 = pods[0]["subpods"][0]["img"]["src"]
        Image(url=image_url2,format='gif')
except Exception as e:
    print(f"wolfram alpha result json structure does not match common structure: {e}")

## References:
- https://python.langchain.com/docs/modules/agents/agent_types/openai_functions_agent
- https://python.langchain.com/docs/integrations/tools/wolfram_alpha
- https://api.python.langchain.com/en/latest/utilities/langchain.utilities.wolfram_alpha.WolframAlphaAPIWrapper.html
- https://api.python.langchain.com/en/latest/_modules/langchain/tools/wolfram_alpha/tool.html
- https://towardsdatascience.com/build-your-next-project-with-wolfram-alpha-api-and-python-51c2c361d8b9
- https://www.packtpub.com/article-hub/unleashing-the-power-of-wolfram-alpha-api-with-python-and-chatgpt
- https://blog.finxter.com/openai-api-functions-embeddings-course-4-7-database-querying-using-chatgpt/
- https://medium.com/mlearning-ai/chatting-with-your-database-using-langchain-e27893eb840a
